In [1]:
import torch
from torch import nn
import utils

In [2]:
# 模型接收size为1 * 224 * 224的单通道图片
net = nn.Sequential(
    # 这里使用一个11*11的更大窗口来捕捉对象。
    # 同时，步幅为4，以减少输出的高度和宽度。
    # 另外，输出通道的数目远大于LeNet
    nn.Conv2d(1, 96, kernel_size=11, stride=4), 
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=3, stride=2),
    # 减小卷积窗口，使用填充为2来使得输入与输出的高和宽一致，且增大输出通道数
    nn.Conv2d(96, 256, kernel_size=5, padding=2), 
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=3, stride=2),
    # 使用三个连续的卷积层和较小的卷积窗口。
    # 除了最后的卷积层，输出通道的数量进一步增加。
    # 在前两个卷积层之后，汇聚层不用于减少输入的高度和宽度
    nn.Conv2d(256, 384, kernel_size=3, padding=1), 
    nn.ReLU(),
    nn.Conv2d(384, 384, kernel_size=3, padding=1), 
    nn.ReLU(),
    nn.Conv2d(384, 256, kernel_size=3, padding=1), 
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=3, stride=2),
    nn.Flatten(),
    # 这里，全连接层的输出数量是LeNet中的好几倍。使用dropout层来减轻过拟合
    nn.Linear(6400, 4096), 
    nn.ReLU(),
    nn.Dropout(p=0.5),
    nn.Linear(4096, 4096), 
    nn.ReLU(),
    nn.Dropout(p=0.5),
    # 最后是输出层。由于这里使用Fashion-MNIST，所以用类别数为10，而非论文中的1000
    nn.Linear(4096, 10))

In [3]:
X = torch.randn(1, 1, 224, 224)
for layer in net:
    X=layer(X)
    print(layer.__class__.__name__,'output shape:\t',X.shape)

Conv2d output shape:	 torch.Size([1, 96, 54, 54])
ReLU output shape:	 torch.Size([1, 96, 54, 54])
MaxPool2d output shape:	 torch.Size([1, 96, 26, 26])
Conv2d output shape:	 torch.Size([1, 256, 26, 26])
ReLU output shape:	 torch.Size([1, 256, 26, 26])
MaxPool2d output shape:	 torch.Size([1, 256, 12, 12])
Conv2d output shape:	 torch.Size([1, 384, 12, 12])
ReLU output shape:	 torch.Size([1, 384, 12, 12])
Conv2d output shape:	 torch.Size([1, 384, 12, 12])
ReLU output shape:	 torch.Size([1, 384, 12, 12])
Conv2d output shape:	 torch.Size([1, 256, 12, 12])
ReLU output shape:	 torch.Size([1, 256, 12, 12])
MaxPool2d output shape:	 torch.Size([1, 256, 5, 5])
Flatten output shape:	 torch.Size([1, 6400])
Linear output shape:	 torch.Size([1, 4096])
ReLU output shape:	 torch.Size([1, 4096])
Dropout output shape:	 torch.Size([1, 4096])
Linear output shape:	 torch.Size([1, 4096])
ReLU output shape:	 torch.Size([1, 4096])
Dropout output shape:	 torch.Size([1, 4096])
Linear output shape:	 torch.Size([1,

In [4]:
lr, num_epochs, batch_size = 0.01, 10, 128
train_iter, test_iter = utils.load_data_fashion_mnist(batch_size, resize=224)
utils.train_ch6(net, train_iter, test_iter, num_epochs, lr, utils.try_gpu())

training on: [cuda:0], [2024-05-30 22:14:08]
epoch: 1/10, loss 1.300, train acc 0.517, test acc 0.723
epoch: 2/10, loss 0.657, train acc 0.754, test acc 0.801
epoch: 3/10, loss 0.537, train acc 0.799, test acc 0.821
epoch: 4/10, loss 0.474, train acc 0.825, test acc 0.840
epoch: 5/10, loss 0.430, train acc 0.842, test acc 0.851
epoch: 6/10, loss 0.400, train acc 0.854, test acc 0.864
epoch: 7/10, loss 0.378, train acc 0.862, test acc 0.868
epoch: 8/10, loss 0.357, train acc 0.869, test acc 0.871
epoch: 9/10, loss 0.342, train acc 0.875, test acc 0.880
epoch: 10/10, loss 0.330, train acc 0.878, test acc 0.884
*** 800.2 examples/sec on cuda:0 - [0:00:12:29], all: [0:00:17:39] ***
